In [1]:
import torch
import numpy as np
import PIL

print(torch.cuda.is_available())

True


In [2]:
!nvidia-smi

Fri Jul 26 19:11:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.06              Driver Version: 545.29.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080 ...    Off | 00000000:01:00.0  On |                  N/A |
| N/A   65C    P8              23W / 125W |    174MiB / 16384MiB |     11%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import pickle
import numpy as np
from skimage import io

from tqdm import tqdm, tqdm_notebook
from PIL import Image
from pathlib import Path

from torchvision import transforms, models, datasets
from multiprocessing.pool import ThreadPool
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

from matplotlib import colors, pyplot as plt
%matplotlib inline

In [4]:
DATA_MODES = ['train', 'val', 'test']
RESCALE_SIZE = 224
DEVICE = torch.device("cuda")

In [5]:
data_transforms = transforms.Compose([
    transforms.Resize((RESCALE_SIZE, RESCALE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

In [6]:
data_dir = "./train/simpsons_dataset"

In [7]:
import math

image_datasets = datasets.ImageFolder(data_dir, data_transforms)

In [8]:
dataset_len = len(image_datasets)
p1 = (dataset_len / 100)
train_size = math.floor(p1 * 70)
test_size = dataset_len - train_size

In [9]:
train_dataset, val_dataset = torch.utils.data.random_split(image_datasets, [train_size, test_size])

In [10]:
len(train_dataset), len(val_dataset)

(14653, 6280)

In [11]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=32,
    shuffle=True
)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=128
)

loaders = {
    "train": train_dataloader,
    "valid": val_dataloader
}

In [12]:
model_resnet18 = models.resnet18(pretrained=True)
model_resnet18

/home/brotiger/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/brotiger/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [13]:
for param in model_resnet18.parameters():
    param.requires_grad = False

In [14]:
num_classes = 42
model_resnet18.fc = nn.Linear(512, num_classes)

In [15]:
import torch.optim as optim

optimizer = optim.Adam(model_resnet18.fc.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, 10, gamma=0.5)

In [16]:
!rm -rf logs

In [17]:
from catalyst import dl

runner = dl.SupervisedRunner()

runner.train(
    model=model_resnet18,
    engine=dl.GPUEngine("cuda"),
    optimizer=optimizer,
    criterion=nn.CrossEntropyLoss(),
    scheduler=scheduler,
    callbacks=[
        dl.CriterionCallback(input_key="logits", target_key="targets", metric_key="loss"),
        dl.BackwardCallback(metric_key="loss"),
        dl.OptimizerCallback(metric_key="loss"), 
        dl.AccuracyCallback(input_key="logits", target_key="targets"),
        dl.SchedulerCallback(),
        dl.PrecisionRecallF1SupportCallback(
            input_key="logits", target_key="targets", num_classes=num_classes, log_on_batch=False
        ),
        dl.MultilabelAccuracyCallback(
            input_key="logits", target_key="targets", threshold=0.5
        )
    ],
    loaders=loaders,
    num_epochs=30,
    verbose=True,
    logdir="logs/resnet18"
)

1/30 * Epoch (train):   0%|          | 0/458 [00:00<?, ?it/s]

train (1/30) accuracy: 0.7991358976083304 | accuracy/std: 0.020360888661742266 | accuracy01: 0.18139630109061586 | accuracy01/std: 0.09364998588199676 | f1/_macro: 0.058930786718675465 | f1/_micro: 0.18139130123656716 | f1/_weighted: 0.15217847325048173 | loss: 2.953725912607887 | loss/mean: 2.953725912607887 | loss/std: 0.2665735935952653 | lr: 0.0001 | momentum: 0.9 | precision/_macro: 0.08816007275263302 | precision/_micro: 0.18139630109875116 | precision/_weighted: 0.18720963525547593 | recall/_macro: 0.06192763475162297 | recall/_micro: 0.18139630109875116 | recall/_weighted: 0.1813963010987511


/home/brotiger/.local/lib/python3.10/site-packages/accelerate/utils/dataclasses.py:480: FutureWarning: The `TPU` of `<enum 'DistributedType'>` is deprecated and will be removed in v1.0.0. Please use the `XLA` instead.
  warnings.warn(


1/30 * Epoch (valid):   0%|          | 0/50 [00:00<?, ?it/s]

valid (1/30) accuracy: 0.831248117404379 | accuracy/std: 0.003964969292878992 | accuracy01: 0.30907643312101923 | accuracy01/std: 0.041505776287375895 | f1/_macro: 0.11204347110001404 | f1/_micro: 0.3090714332019039 | f1/_weighted: 0.27175528329094323 | loss: 2.6657717103411436 | loss/mean: 2.6657717103411436 | loss/std: 0.09970883651796326 | lr: 0.0001 | momentum: 0.9 | precision/_macro: 0.17247337470495808 | precision/_micro: 0.30907643312101907 | precision/_weighted: 0.36115306379038503 | recall/_macro: 0.11603839395954878 | recall/_micro: 0.30907643312101907 | recall/_weighted: 0.3090764331210191
* Epoch (1/30) lr: 0.0001 | momentum: 0.9


...

valid (29/30) accuracy: 0.9843456307034585 | accuracy/std: 0.0014276826545724169 | accuracy01: 0.7406050955414013 | accuracy01/std: 0.03729755171245238 | f1/_macro: 0.45812057770930553 | f1/_micro: 0.7406000955751572 | f1/_weighted: 0.7259783619691564 | loss: 1.0508175966845956 | loss/mean: 1.0508175966845956 | loss/std: 0.09931092501801708 | lr: 2.5e-05 | momentum: 0.9 | precision/_macro: 0.6109764025718439 | precision/_micro: 0.7406050955414012 | precision/_weighted: 0.7374004275076405 | recall/_macro: 0.4257814398324252 | recall/_micro: 0.7406050955414012 | recall/_weighted: 0.7406050955414013
* Epoch (29/30) lr: 2.5e-05 | momentum: 0.9


30/30 * Epoch (train):   0%|          | 0/458 [00:00<?, ?it/s]

train (30/30) accuracy: 0.985130634082834 | accuracy/std: 0.003207108028168378 | accuracy01: 0.7441479560743626 | accuracy01/std: 0.0770751193758001 | f1/_macro: 0.5019319574638453 | f1/_micro: 0.7441429560835509 | f1/_weighted: 0.7315506352378041 | loss: 1.023387267013089 | loss/mean: 1.023387267013089 | loss/std: 0.1916608901988916 | lr: 2.5e-05 | momentum: 0.9 | precision/_macro: 0.735580640321125 | precision/_micro: 0.7441479560499557 | precision/_weighted: 0.7486636800174528 | recall/_macro: 0.45869012834720513 | recall/_micro: 0.7441479560499557 | recall/_weighted: 0.7441479560499557


30/30 * Epoch (valid):   0%|          | 0/50 [00:00<?, ?it/s]

valid (30/30) accuracy: 0.9847247642316634 | accuracy/std: 0.0014221868543398018 | accuracy01: 0.7420382165605093 | accuracy01/std: 0.038366336350175244 | f1/_macro: 0.4516234506766945 | f1/_micro: 0.7420332165942002 | f1/_weighted: 0.7268045860114004 | loss: 1.0476271024934802 | loss/mean: 1.0476271024934802 | loss/std: 0.10264077682426327 | lr: 2.5e-05 | momentum: 0.9 | precision/_macro: 0.6049070151903648 | precision/_micro: 0.7420382165605095 | precision/_weighted: 0.7373464132455082 | recall/_macro: 0.4218579394833971 | recall/_micro: 0.7420382165605095 | recall/_weighted: 0.7420382165605095
* Epoch (30/30) lr: 1.25e-05 | momentum: 0.9
Top models:
logs/resnet18/checkpoints/model.0030.pth	30.0000


In [18]:
%load_ext tensorboard
%tensorboard --logdir logs